In [5]:
import pandas as pd
import numpy as np
data=pd.read_csv("/sipri-arms-by-seller-all.csv")


In [6]:
data.head()

,tidn,buyercod,sellercod,odat,odai,onum,onai,ldat,term,desig2,...,coprod,nrdel,nrdelai,delyears,buyer,seller,status,tivunit,tivorder,tivdel
0,55238.0,BF,ALB,2011.0,X,12.0,X,2011.0,NaN,M-43 120mm,...,L,12.0,NaN,2011,Burkina Faso,Albania,S,0.020,0.240,0.240
1,3622.0,ANG,ALG,1980.0,NaN,4.0,NaN,1980.0,NaN,N-262 Fregate,...,L,4.0,NaN,1980,Angola,Algeria,S,1.500,6.000,6.000
2,11246.0,NIG,ALG,1967.0,NaN,2.0,X,1968.0,NaN,Il-28,...,L,2.0,X,1968,Nigeria,Algeria,S,2.880,5.760,5.760
3,3667.0,CON,ANG,1979.0,NaN,1.0,NaN,1979.0,AID,Noratlas,...,L,1.0,NaN,1979,Congo,Angola,S,2.800,2.800,2.800
4,3684.0,GBI,ANG,1980.0,NaN,1.0,NaN,1980.0,NaN,Do-27,...,L,1.0,NaN,1980,Guinea-Bissau,Angola,S,0.048,0.048,0.048


In [125]:
data['buyer'].replace(['Soviet Union'], 
    value='Russia',inplace=True)
data['seller'].replace(['Soviet Union'], 
    value='Russia',inplace=True)


country_names=pd.unique(data[['seller', 'buyer']].values.ravel())
country_names=country_names[0:257]
years=[x+1946 for x in range(74)]


In [188]:
def adjacency_year(data,year,country_names):
    temp_dat=data.loc[data['odat'] == year]
    adjacency=np.zeros(len(country_names)**2).reshape(len(country_names),len(country_names))
    adjacency=pd.DataFrame(adjacency)
    adjacency.columns=country_names
    adjacency.index=country_names

    for i in range(temp_dat.shape[0]):
        adjacency.ix[temp_dat['seller'].iloc[i],temp_dat['buyer'].iloc[i]]=1
    adjacency=np.array(adjacency)
    adjacency=adjacency+adjacency.T
    adjacency[adjacency>0]=1
    adjacency=adjacency+np.diag(np.ones(len(country_names)))
    return(adjacency)

def adjacency_year_between(data,year,country_names):
    temp_dat=data.iloc[np.where((year>=data['odat'])*1 *( data['ldat']>=year)*1)[0]]
    adjacency=np.zeros(len(country_names)**2).reshape(len(country_names),len(country_names))
    adjacency=pd.DataFrame(adjacency)
    adjacency.columns=country_names
    adjacency.index=country_names

    for i in range(temp_dat.shape[0]):
        adjacency.ix[temp_dat['seller'].iloc[i],temp_dat['buyer'].iloc[i]]=1
    adjacency=np.array(adjacency)
    adjacency=adjacency+adjacency.T
    adjacency[adjacency>0]=1
    adjacency=adjacency+np.diag(np.ones(len(country_names)))
    return(adjacency)

def create_pairs(x,y):
    pairs=[]
    labels=[]
    for q in range(len(y)):
        temp_x=x[q].copy()
        temp_y=y[q].copy()
        for i in range(len(temp_y)):
            for j in range(len(temp_y)-i):
                j=j+i
                pairs += [[temp_x[i],temp_x[j]]] 
                labels += [temp_y[i,j]]
    return(pairs,labels)


In [189]:
y_mat=[adjacency_year(data,x,country_names) for x in years]
y_mat_between=[adjacency_year_between(data,x,country_names) for x in years]

In [190]:
x_mat=[np.column_stack((np.diag(np.ones(len(country_names))),(np.ones(len(country_names))*x-36.5)/22 )) for x in range(len(years))]


In [196]:
pairs, labels=create_pairs(x_mat,y_mat_between)
pairs=np.array(pairs)
labels=np.array(labels)


In [197]:
import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

input_dim=258
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1-y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(.1))
    seq.add(Dense(128,activation='relu'))
    seq.add(Dropout(.1))
    seq.add(Dense(128,activation='relu'))

    seq.add(Dense(2))
    return seq


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


# the data, shuffled and split between train and test sets

# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([pairs[:, 0], pairs[:, 1]], labels,
          batch_size=10240,
          epochs=25)

embeddings=Model([input_a], [processed_a])
embeddings_all=embeddings.predict(np.row_stack(x_train_1))



Epoch 1/25
2453322/2453322 [==============================] - 111s - loss: 0.0579   
Epoch 2/25
2453322/2453322 [==============================] - 95s - loss: 0.0279    
Epoch 3/25
2453322/2453322 [==============================] - 112s - loss: 0.0260   
Epoch 4/25
2453322/2453322 [==============================] - 104s - loss: 0.0249   
Epoch 5/25
2453322/2453322 [==============================] - 94s - loss: 0.0241    
Epoch 6/25
2453322/2453322 [==============================] - 91s - loss: 0.0233    
Epoch 7/25
2453322/2453322 [==============================] - 91s - loss: 0.0229    
Epoch 8/25
2453322/2453322 [==============================] - 91s - loss: 0.0225    
Epoch 9/25
2453322/2453322 [==============================] - 90s - loss: 0.0222    
Epoch 10/25
2453322/2453322 [==============================] - 90s - loss: 0.0220    
Epoch 11/25
2453322/2453322 [==============================] - 89s - loss: 0.0217    
Epoch 12/25
2453322/2453322 [==============================] - 

In [198]:
from sklearn.manifold import SpectralEmbedding
tsne_model = SpectralEmbedding(n_components=2,affinity='rbf')
np.set_printoptions(suppress=True)
output=tsne_model.fit_transform(embeddings_all)


In [199]:

pd.DataFrame(embeddings_all).to_csv('/users/sweiss/downloads/embedings_multiyear.csv')
pd.DataFrame(output).to_csv('/users/sweiss/downloads/spectral_embbedings_multiyear.csv')


In [201]:
pd.DataFrame(country_names).to_csv('/users/sweiss/downloads/embeddings_countrynames.csv')



In [208]:

pd.DataFrame(np.row_stack(np.array(y_mat))).to_csv('/users/sweiss/downloads/network_singleyear.csv')
pd.DataFrame(np.row_stack(np.array(y_mat_between))).to_csv('/users/sweiss/downloads/network_multiyear.csv')
